In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess

from datetime import date

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.colors as colors

USER = os.environ['USER']
os.environ['CESMDATAROOT'] = f'/glade/scratch/{USER}/inputdata'
import pop_tools

import regrid_tools
import util3
from glob import glob
import util2

### Regrids x1 transient Nitrogen deposition dataset to x0.1

## **MUST RUN WITH LARGE MEMORY: ~360GB!**

In [3]:
src_grid = 'POP_gx1v7'
dst_grid = 'POP_tx0.1v3'
method = 'conserve'

clobber = True

get_dst_grid = pop_tools.get_grid

In [4]:
def get_regridder(src_grid, dst_grid, method):

    os.makedirs('data/regridding', exist_ok=True)
    dst_grid_file= f'data/regridding/{dst_grid}.nc'
    src_grid_file = f'data/regridding/{src_grid}.nc'
    weight_file = f'data/regridding/{src_grid}_to_{dst_grid}_{method}.nc'

    if not os.path.exists(src_grid_file) or clobber:
        dso = get_dst_grid(src_grid, scrip=True)
        print(f'writing {src_grid_file}')
        dso['grid_imask'].data[:]=1
        print(dso)
        dso.to_netcdf(src_grid_file)    
        
    if not os.path.exists(dst_grid_file) or clobber:
        dso = get_dst_grid(dst_grid, scrip=True)
        print(f'writing {dst_grid_file}')
        dso
        dso.to_netcdf(dst_grid_file)    
        
    if not os.path.exists(weight_file) or clobber:
        cmd = ['ESMF_RegridWeightGen', '--netcdf4', '--ignore_unmapped',
                    '-s', src_grid_file, '-d', dst_grid_file, '-m', method, '-w', weight_file]
        out = subprocess.run(cmd, capture_output=True, check=True)
        print(out.stdout.decode('UTF-8'))
        
    return util2.regridder(src_grid_file, dst_grid_file, weight_file)        

regrid_op = get_regridder(src_grid, dst_grid, method)
regrid_op

/glade/work/kristenk/miniconda/envs/analysis5/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 6103. The TBB threading layer is disabled.
  warnings.warn(problem)


writing data/regridding/POP_gx1v7.nc
<xarray.Dataset>
Dimensions:          (grid_corners: 4, grid_rank: 2, grid_size: 122880, nreg: 13)
Coordinates:
  * nreg             (nreg) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: grid_corners, grid_rank, grid_size
Data variables:
    grid_dims        (grid_rank) int32 320 384
    grid_center_lat  (grid_size) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    grid_center_lon  (grid_size) float64 -39.44 -38.31 -37.19 ... -40.65 -40.22
    grid_corner_lat  (grid_size, grid_corners) float64 -78.95 -78.95 ... 71.96
    grid_corner_lon  (grid_size, grid_corners) float64 321.1 320.0 ... 320.0
    grid_imask       (grid_size) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    region_name      (nreg) <U21 'Black Sea' 'Baltic Sea' ... 'Hudson Bay'
    region_val       (nreg) int64 -13 -12 -5 1 2 3 4 6 7 8 9 10 11
Attributes:
    lateral_dims:       [384, 320]
    vertical_dims:      60
    vert_grid_file:     gx1v7_vert_grid
    hor

regridder POP_gx1v7.nc --> POP_tx0.1v3.nc

In [5]:
file_in = '/glade/p/cesmdata/cseg/inputdata/ocn/pop/gx1v6/forcing/ndep_ocn_SMYLE_w_nhx_emis_gx1v6_1652-2025_c201113.nc'

In [6]:
ds_src = xr.open_dataset(file_in).rename({'Y':'nlat', 'X':'nlon'})  #,chunks={'time': 4})
ds_src.info()

xarray.Dataset {
dimensions:
	d2 = 2 ;
	nlat = 384 ;
	nlon = 320 ;
	time = 4488 ;

variables:
	int32 KMT(nlat, nlon) ;
		KMT:units = unitless ;
		KMT:long_name = k Index of Deepest Grid Cell on T Grid ;
	float32 NHx_deposition(time, nlat, nlon) ;
		NHx_deposition:units = kg(N)/m2/s ;
		NHx_deposition:long_name = NHx deposition ;
		NHx_deposition:cell_methods = time: mean ;
	float32 NOy_deposition(time, nlat, nlon) ;
		NOy_deposition:units = kg(N)/m2/s ;
		NOy_deposition:long_name = NOy deposition ;
		NOy_deposition:cell_methods = time: mean ;
	int32 REGION_MASK(nlat, nlon) ;
		REGION_MASK:units = Basin Index ;
		REGION_MASK:long_name = basin index number (signed integers) ;
	float32 TAREA(nlat, nlon) ;
		TAREA:units = centimeter^2 ;
		TAREA:long_name = area of T cells ;
	float32 TLAT(nlat, nlon) ;
		TLAT:units = degrees_north ;
		TLAT:long_name = Latitude (T grid) ;
	float32 TLONG(nlat, nlon) ;
		TLONG:units = degrees_east ;
		TLONG:long_name = Longitude (T grid) ;
	float32 ULAT(nlat, 

In [7]:
ds_out = pop_tools.get_grid('POP_tx0.1v3')

### Generate masks

In [8]:
# generate 3D grid masks

def gen_MASK(ds_grid):
    nj, ni = ds_grid.KMT.shape

    ONES_2d = xr.DataArray(np.ones((nj, ni)), dims=('nlat', 'nlon'))

    # mask out cells where k is below KMT
    MASK = ONES_2d.where(ONES_2d < ds_grid.KMT)
    MASK = xr.where(MASK.notnull(), True, False)

    return MASK

MASK_out = gen_MASK(ds_out)
(nj_out, ni_out) = MASK_out.shape

grid_refcase = pop_tools.get_grid(grid_name='POP_gx1v7')
MASK_refcase = gen_MASK(grid_refcase)

(nj_refcase, ni_refcase) = MASK_refcase.shape

In [9]:
np.sum(1-MASK_out)

<xarray.DataArray ()>
array(3237440)

In [10]:
ds_grid = pop_tools.get_grid('POP_gx1v7')

nk = len(ds_grid.z_t)
nj, ni = ds_grid.KMT.shape
ONES_2d = xr.DataArray(np.ones((nj, ni)), dims=('nlat', 'nlon'))
MASK = xr.where(ONES_2d.notnull(), True, False)

In [11]:
xlen=len(ds_grid.nlon)
ylen=len(ds_grid.nlat)
print(xlen,ylen)

320 384


### Cycle through time dimension by year and write out each file

In [12]:
tlen = len(ds_src.time)
tlen

4488

In [13]:
# %%time

# vars = ['NHx_deposition','NOy_deposition']

# year = 1 #there are 374 years total

# for j_ts  in np.arange(0,4477,12): #np.arange(0,25,12):
    
#     ds_dst_xy = xr.Dataset()
#     ds_year = ds_src.isel(time=slice(j_ts,j_ts+12))
    
#     for v in vars:

#         print('year ',year,': starting loop for', v)

        
#         ## do lateral fill
#         for m in np.arange(0,12,1):
    
#             da_tmp = ds_year[v].isel(time=m)
#             ds_year[v][m,:,:] = pop_tools.lateral_fill(da_tmp, MASK, ltripole=False, use_sor=True, max_iter=1000)
    
#         ds_dst_xy[v] = regrid_op.regrid_dataarray( ds_year[v],renormalize=True, apply_mask=False) #changed from True
        
#     file_name = '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year' + str(year) + '.nc'
    
#     ds_dst_xy.to_netcdf(file_name, mode='w')
    
#     year = year + 1

### check and make sure there are the right number of nans

In [14]:
np.sum(1-MASK_out)

<xarray.DataArray ()>
array(3237440)

In [80]:
#np.sum(np.isnan(ds_dst_xy.NOy_deposition.isel(time=0)))

<xarray.DataArray 'NOy_deposition' ()>
array(3237440)
Coordinates:
    time     object 1654-01-16 12:00:00

### Now read the files back in (in the correct order) and concat

In [15]:
files = []
for year in range(1,375): #range(1,4):
    syr=str(year)
    #print(year)
    files.extend(sorted(glob(f'/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year{syr}.nc')))

In [16]:
files

['/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year1.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year2.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year3.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year4.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year5.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year6.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year7.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year8.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year9.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year10.nc',
 '/glade/scratch/kristenk/cesm_inputdata/N_dep_each_year/hi_res_Ndep_forcing_year11.nc',
 '/glade/scratch/kristenk/cesm

In [17]:
ds_out=xr.open_mfdataset(files,decode_times=False,decode_coords=False, concat_dim='time')

In [18]:
ds_out

<xarray.Dataset>
Dimensions:         (nlat: 2400, nlon: 3600, time: 4488)
Coordinates:
  * time            (time) float64 -7.189e+04 -7.186e+04 ... 6.456e+04 6.459e+04
Dimensions without coordinates: nlat, nlon
Data variables:
    NHx_deposition  (time, nlat, nlon) float64 dask.array<chunksize=(12, 2400, 3600), meta=np.ndarray>
    NOy_deposition  (time, nlat, nlon) float64 dask.array<chunksize=(12, 2400, 3600), meta=np.ndarray>

In [19]:
ds_out['time'] = ds_src.time

In [20]:
ds_out.time

<xarray.DataArray 'time' (time: 4488)>
array([cftime.DatetimeNoLeap(1652, 1, 16, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1652, 2, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1652, 3, 16, 12, 0, 0, 0), ...,
       cftime.DatetimeNoLeap(2025, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(2025, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(2025, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1652-01-16 12:00:00 ... 2025-12-16 12:00:00
Attributes:
    long_name:  time
    bounds:     time_bnds

In [21]:
import util

### change coord variables to X and Y and add a few more variables so it's exactly like the x1 transient Ndep file

In [22]:
ds_ndep_clim = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/ocn/pop/tx0.1v3/forcing/ndep_ocn_1850_w_nhx_emis_tx0.1v3_c191115.nc')

In [23]:
ds_out = ds_out.rename({'nlat':'Y', 'nlon':'X'}) 

In [24]:
ds_out['KMT'] = ds_ndep_clim['KMT']
ds_out['REGION_MASK'] = ds_ndep_clim['REGION_MASK']
ds_out['TAREA'] = ds_ndep_clim['TAREA']
ds_out['TLAT'] = ds_ndep_clim['TLAT']
ds_out['TLONG'] = ds_ndep_clim['TLONG']
ds_out['ULAT'] = ds_ndep_clim['ULAT']
ds_out['ULONG'] = ds_ndep_clim['ULONG']
ds_out['X'] = ds_ndep_clim['X']
ds_out['Y'] = ds_ndep_clim['Y']

In [25]:
ds_out

<xarray.Dataset>
Dimensions:         (X: 3600, Y: 2400, time: 4488)
Coordinates:
  * Y               (Y) float32 1.0 2.0 3.0 4.0 ... 2.398e+03 2.399e+03 2.4e+03
  * X               (X) float32 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * time            (time) object 1652-01-16 12:00:00 ... 2025-12-16 12:00:00
    TLAT            (Y, X) float32 ...
    TLONG           (Y, X) float32 ...
Data variables:
    NHx_deposition  (time, Y, X) float64 dask.array<chunksize=(12, 2400, 3600), meta=np.ndarray>
    NOy_deposition  (time, Y, X) float64 dask.array<chunksize=(12, 2400, 3600), meta=np.ndarray>
    KMT             (Y, X) int32 ...
    REGION_MASK     (Y, X) int32 ...
    TAREA           (Y, X) float32 ...
    ULAT            (Y, X) float32 ...
    ULONG           (Y, X) float32 ...

In [26]:
ds_ndep_clim

<xarray.Dataset>
Dimensions:         (X: 3600, Y: 2400, time: 12)
Coordinates:
  * time            (time) object 1850-01-15 00:00:00 ... 1850-12-15 00:00:00
  * X               (X) float32 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * Y               (Y) float32 1.0 2.0 3.0 4.0 ... 2.398e+03 2.399e+03 2.4e+03
    TLAT            (Y, X) float32 ...
    TLONG           (Y, X) float32 ...
Data variables:
    ULAT            (Y, X) float32 ...
    ULONG           (Y, X) float32 ...
    TAREA           (Y, X) float32 ...
    REGION_MASK     (Y, X) int32 ...
    KMT             (Y, X) int32 ...
    NOy_deposition  (time, Y, X) float32 ...
    NHx_deposition  (time, Y, X) float32 ...

In [27]:
ds_out.NOy_deposition

<xarray.DataArray 'NOy_deposition' (time: 4488, Y: 2400, X: 3600)>
dask.array<concatenate, shape=(4488, 2400, 3600), dtype=float64, chunksize=(12, 2400, 3600), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float32 1.0 2.0 3.0 4.0 ... 2.398e+03 2.399e+03 2.4e+03
  * X        (X) float32 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * time     (time) object 1652-01-16 12:00:00 ... 2025-12-16 12:00:00
    TLAT     (Y, X) float32 ...
    TLONG    (Y, X) float32 ...
Attributes:
    units:         kg(N)/m2/s
    long_name:     NOy deposition
    cell_methods:  time: mean

In [28]:
datestamp = date.today().strftime("%y%m%d")
ds_out.attrs['history'] = f'created by Kristen Krumhardt on {datestamp}'
file_out = '/glade/scratch/kristenk/cesm_inputdata/ocn_Ndep_transient_forcing_x0.1_'+datestamp+'.nc'
ds_out.attrs['input_file_list'] = file_in
util.ds_clean(ds_out).to_netcdf(file_out, unlimited_dims='time') #,format="NETCDF3_64BIT")

In [29]:
from subprocess import Popen, PIPE

In [30]:
def nco(cmd):
    """Interface to NCO"""
    p = Popen(
        ' && '.join(['module load nco', ' '.join(cmd)]),
        stdout=PIPE,
        stderr=PIPE,
        shell=True
    )

    stdout, stderr = p.communicate()
    if p.returncode != 0:
        print(stdout.decode('UTF-8'))
        print(stderr.decode('UTF-8'))
        raise

In [31]:
print("reformatting file")
nco(["ncks", "-O", "-5", file_out, file_out])
print("done")

reformatting file
done


In [33]:
#ds_out.NOy_deposition.isel(time=0)
np.sum(np.isnan(ds_out.NOy_deposition.isel(time=0).values))

3237440